# Generating Qubit Hamiltonians

In [4]:
import tequila as tq
from utility import *
import copy
P_X = np.array([[0, 1], [1, 0]])
P_Y = np.array([[0, 0 -1j],[0+1j,0]])
P_Z = np.array([[1, 0], [0, -1]])

Specify the Qubit Hamiltonian of a molecule by its name, internuclear distances, basis set, and fermion-to-qubit transformation.
Here, we show the resulting Hamiltonian for $H_2$ in STO-3G with $1\overset{\circ}{A}$ between the $H$ atoms. 

In [5]:
qubit_transf = 'jw' # Jordan-Wigner transformations
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print(h2)


#Get additional information from tequila
xyz_data = get_molecular_data('h2', geometry=2.5, xyz_format=True)
basis='sto-3g'
h2_tq = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)


-0.32760818967480887 [] +
-0.04919764587136755 [X0 X1 Y2 Y3] +
0.04919764587136755 [X0 Y1 Y2 X3] +
0.04919764587136755 [Y0 X1 X2 Y3] +
-0.04919764587136755 [Y0 Y1 X2 X3] +
0.13716572937099497 [Z0] +
0.15660062488237947 [Z0 Z1] +
0.10622904490856075 [Z0 Z2] +
0.15542669077992832 [Z0 Z3] +
0.13716572937099492 [Z1] +
0.15542669077992832 [Z1 Z2] +
0.10622904490856075 [Z1 Z3] +
-0.13036292057109117 [Z2] +
0.16326768673564346 [Z2 Z3] +
-0.13036292057109117 [Z3]


Alternatively, the qubit-tapering technique can find a smaller effective Hamitlonian by subsitituting operators with $\pm 1$. This technique is detailed in Bravyi's work ([Bravyi et al., "Tapering off qubits to simulate fermionic Hamiltonians", arXiv:1701.08213](https://arxiv.org/abs/1701.08213)). 

In [6]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h2, n_spin_orbitals=2*h2_tq.n_orbitals, n_electrons=h2_tq.n_electrons, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -0.5310513494337641 [] +
0.1967905834854702 [X0] +
-0.5350572998841723 [Z0]


We can verify that this new Hamiltonian indeed includes the ground state. 

In [7]:
print("The ground state energy:")
obtain_PES('h2', [1], 'sto-3g', 'fci')

# Building the matrix representation of the effective Hamiltonian
I, X, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[1, 0], [0, -1]])
h2_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h2_matrix)
print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

The ground state energy:
E = -1.1011503301329566 Eh

The eigenvalues in the effective Hamiltonian: 
 [-1.10115031  0.03904763]


# H<sub>2</sub>O
Let's do the same for H<sub>2</sub>O, this will be a much bigger matrix

In [9]:
qubit_transf = "jw"
h2o = get_qubit_hamiltonian(mol='h2o', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print(h2o)

-46.577441376239165 [] +
-0.01475406443554861 [X0 X1 Y2 Y3] +
0.005507932036999009 [X0 X1 Y2 Z3 Z4 Z5 Z6 Y7] +
-0.00860598144780857 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
0.005507932036999009 [X0 X1 X3 Z4 Z5 X6] +
-0.00860598144780857 [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
-0.0026730909100343733 [X0 X1 Y4 Y5] +
-0.0037537905436213223 [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Z11 Z12 Y13] +
-0.0037537905436213223 [X0 X1 X5 Z6 Z7 Z8 Z9 Z10 Z11 X12] +
-0.006698936099693298 [X0 X1 Y6 Y7] +
-9.23430100743497e-05 [X0 X1 Y6 Z7 Z8 Z9 Z10 Y11] +
-9.23430100743497e-05 [X0 X1 X7 Z8 Z9 X10] +
-0.006505428251122429 [X0 X1 Y8 Y9] +
-0.007380770580843395 [X0 X1 Y10 Y11] +
-0.005283643032839312 [X0 X1 Y12 Y13] +
0.01475406443554861 [X0 Y1 Y2 X3] +
-0.005507932036999009 [X0 Y1 Y2 Z3 Z4 Z5 Z6 X7] +
0.00860598144780857 [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
0.005507932036999009 [X0 Y1 Y3 Z4 Z5 X6] +
-0.00860598144780857 [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
0.0026730909100343733 [X0 Y1 Y4 X5] +
0.0037537905436213223 [

In [10]:
#Get additional information from tequila
xyz_data = get_molecular_data('h2o', geometry=1, xyz_format=True)
basis='sto-3g'
h2o_tq = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

#Taper Hamiltonian
h2o_small = taper_hamiltonian(h2o, n_spin_orbitals=2*h2o_tq.n_orbitals, n_electrons=h2o_tq.n_electrons, qubit_transf=qubit_transf)
print("The effective Hamiltonian:\n {}".format(h2o_small)) 

The effective Hamiltonian:
 -46.3574016028954 [] +
0.10505651008352154 [X0] +
-0.011299006073352824 [X0 X1 X2 Z3 Z4 Z6 Z7 X8] +
0.03522559376822524 [X0 X1 X2 Z8 Z9] +
0.0021883512242770473 [X0 X1 Z2 Z3 X4 Z5 Z7 Z8] +
-0.001664743498854376 [X0 X1 Z2 Z3 Z5 Z6 X7 Z8] +
-0.03172875970749076 [X0 X1 Y3 Y4] +
-0.005996185758110515 [X0 X1 Y3 Z4 Z6 Y7] +
0.011299006073352826 [X0 X1 Z3 Z4 Z6 Z7 X9] +
-0.005996185758110515 [X0 X1 X4 X6] +
0.0021883512242770473 [X0 X1 X4 Z6 Z7 Z8 Z9] +
0.03654446580389959 [X0 X1 X5] +
-0.024829490206145624 [X0 X1 Y6 Y7] +
-0.001664743498854376 [X0 X1 X7 Z8 Z9] +
-0.015996127256195754 [X0 X1 Y8 Y9] +
-0.011299006073352824 [X0 Y1 Y2 Z3 Z4 Z6 Z7 X8] +
0.03522559376822524 [X0 Y1 Y2 Z8 Z9] +
-0.0021883512242770473 [X0 Y1 Z2 Y3 Z4 Z5 Z7 Z8] +
0.0021883512242770473 [X0 Y1 Z2 Z3 Y4 Z5 Z7 Z8] +
0.0016647434988543758 [X0 Y1 Z2 Z3 Z5 Y6 Z7 Z8] +
-0.001664743498854376 [X0 Y1 Z2 Z3 Z5 Z6 Y7 Z8] +
0.03172875970749076 [X0 Y1 Y3 X4] +
0.005996185758110515 [X0 Y1 Y3 Z4 Z6 X7] +
-0

Looks like it changed from a 14 qubit Hamiltonian to a 10 qubit one. Oh well,every bit helps!
In order to help out with the calculations for H<sub>2</sub>O (and other molecules) we write a function that will help 'stitch' together a Hamiltonian Matrix from the openfermion object

In [11]:
#Convert to string, we'll parse it that way
h2o_small_str = format(h2o_small)
n_qubits = 10 # For H20

def str2mat(hamiltonian_string,n_qubits):
    total_hamiltonian = None  #stores the total hamiltonian
    first_flag = True
    
    for line in hamiltonian_string.splitlines():
        temp_hamiltonian = None #stores the temporary hamiltonian
        matrix_operand = None
        var_start = line.find('[')
        var_end = line.find(']')
        coeff = float(line[0:var_start]) #coefficients for individual parts
        temp_ham_set = line[var_start+1:var_end].split() #variables for individual parts
        
        indiv_ham_set = ['I' for i in range(0,n_qubits)] #this decides what operator affects which qubit, initialize to identity
        
        if len(temp_ham_set)!=0:
            for operator in temp_ham_set:
                q_index = int(operator[1:len(str(n_qubits))+1]) #Find out which qubit it acts on
                if operator[0]=='X':
                    indiv_ham_set[q_index] = 'P_X'
                elif operator[0]=='Y':
                    indiv_ham_set[q_index] = 'P_Y'
                elif operator[0]=='Z':
                    indiv_ham_set[q_index] = 'P_Z'
        #print(temp_ham_set)
        #print(indiv_ham_set)
        
        for i in range(0,n_qubits-1):
            if i== 0:
                #reinitialize temp_hamiltonian from first qubit
                if indiv_ham_set[i] == 'I':
                    temp_hamiltonian = copy.deepcopy(np.eye(2))
                elif indiv_ham_set[i] == 'P_X':
                    temp_hamiltonian = copy.deepcopy(P_X)
                elif indiv_ham_set[i] == 'P_Y':
                    temp_hamiltonian = copy.deepcopy(P_Y)
                elif indiv_ham_set[i] == 'P_Z':
                    temp_hamiltonian = copy.deepcopy(P_Z)
            #now for the operand to which the temp_hamiltonian will be kroneckered
            if indiv_ham_set[i+1] == 'I':
                matrix_operand = copy.deepcopy(np.eye(2))
            elif indiv_ham_set[i+1] == 'P_X':
                matrix_operand = copy.deepcopy(P_X)
            elif indiv_ham_set[i+1] == 'P_Y':
                matrix_operand = copy.deepcopy(P_Y)
            elif indiv_ham_set[i+1] == 'P_Z':
                matrix_operand = copy.deepcopy(P_Z)
                
            #Kronecker together
            temp_hamiltonian = np.kron(temp_hamiltonian,matrix_operand)

        if first_flag == True:
            temp_hamiltonian = coeff*temp_hamiltonian
            total_hamiltonian = copy.deepcopy(temp_hamiltonian)
            first_flag = False
            #print(total_hamiltonian)
        else:
            total_hamiltonian = total_hamiltonian + coeff*temp_hamiltonian
    
    return total_hamiltonian

h2o_ham_mat = str2mat(h2o_small_str,n_qubits)

eigvals, _ = np.linalg.eigh(h2o_ham_mat)
print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

print("The best one (ground state) is :",eigvals[0])


The eigenvalues in the effective Hamiltonian: 
 [-75.0176887  -74.5705327  -74.46690758 ...  -2.3782076    0.7867031
   1.32218701]
The best one (ground state) is : -75.01768869618216


# LiH
Taking it down a notch, its time for LiH

In [12]:
qubit_transf = "jw"
lih = get_qubit_hamiltonian(mol='lih', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print(lih)

-3.934441956757907 [] +
-0.007923321157850459 [X0 X1 Y2 Y3] +
-0.0034145323580157642 [X0 X1 Y2 Z3 Z4 Y5] +
-0.00274686132031707 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.0034145323580157642 [X0 X1 X3 X4] +
-0.00274686132031707 [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
-0.004864778381761084 [X0 X1 Y4 Y5] +
-0.002296316587299456 [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.002296316587299456 [X0 X1 X5 Z6 Z7 Z8 Z9 X10] +
-0.00247270616838528 [X0 X1 Y6 Y7] +
-0.0024727061683852813 [X0 X1 Y8 Y9] +
-0.001774435009950255 [X0 X1 Y10 Y11] +
0.007923321157850459 [X0 Y1 Y2 X3] +
0.0034145323580157642 [X0 Y1 Y2 Z3 Z4 X5] +
0.00274686132031707 [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.0034145323580157642 [X0 Y1 Y3 X4] +
-0.00274686132031707 [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
0.004864778381761084 [X0 Y1 Y4 X5] +
0.002296316587299456 [X0 Y1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.002296316587299456 [X0 Y1 Y5 Z6 Z7 Z8 Z9 X10] +
0.00247270616838528 [X0 Y1 Y6 X7] +
0.0024727061683852813 [X0 Y1 Y8 X9] +
0.0017744350099

In [17]:
#Get additional information from tequila
xyz_data = get_molecular_data('lih', geometry=1, xyz_format=True)
basis='sto-3g'
lih_tq = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

#Taper Hamiltonian
lih_small = taper_hamiltonian(lih, n_spin_orbitals=2*lih_tq.n_orbitals, n_electrons=lih_tq.n_electrons, qubit_transf=qubit_transf)
print("The effective Hamiltonian:\n {}".format(h2o_small)) 

The effective Hamiltonian:
 -46.3574016028954 [] +
0.10505651008352154 [X0] +
-0.011299006073352824 [X0 X1 X2 Z3 Z4 Z6 Z7 X8] +
0.03522559376822524 [X0 X1 X2 Z8 Z9] +
0.0021883512242770473 [X0 X1 Z2 Z3 X4 Z5 Z7 Z8] +
-0.001664743498854376 [X0 X1 Z2 Z3 Z5 Z6 X7 Z8] +
-0.03172875970749076 [X0 X1 Y3 Y4] +
-0.005996185758110515 [X0 X1 Y3 Z4 Z6 Y7] +
0.011299006073352826 [X0 X1 Z3 Z4 Z6 Z7 X9] +
-0.005996185758110515 [X0 X1 X4 X6] +
0.0021883512242770473 [X0 X1 X4 Z6 Z7 Z8 Z9] +
0.03654446580389959 [X0 X1 X5] +
-0.024829490206145624 [X0 X1 Y6 Y7] +
-0.001664743498854376 [X0 X1 X7 Z8 Z9] +
-0.015996127256195754 [X0 X1 Y8 Y9] +
-0.011299006073352824 [X0 Y1 Y2 Z3 Z4 Z6 Z7 X8] +
0.03522559376822524 [X0 Y1 Y2 Z8 Z9] +
-0.0021883512242770473 [X0 Y1 Z2 Y3 Z4 Z5 Z7 Z8] +
0.0021883512242770473 [X0 Y1 Z2 Z3 Y4 Z5 Z7 Z8] +
0.0016647434988543758 [X0 Y1 Z2 Z3 Z5 Y6 Z7 Z8] +
-0.001664743498854376 [X0 Y1 Z2 Z3 Z5 Z6 Y7 Z8] +
0.03172875970749076 [X0 Y1 Y3 X4] +
0.005996185758110515 [X0 Y1 Y3 Z4 Z6 X7] +
-0

In [18]:
lih_small_str = format(lih_small)
n_qubits = 10 # For LiH


lih_ham_mat = str2mat(lih_small_str,n_qubits)

eigvals, _ = np.linalg.eigh(lih_ham_mat)
print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

print("The best one (ground state) is :",eigvals[0])


The eigenvalues in the effective Hamiltonian: 
 [-7.78446028 -7.78446028 -7.78446028 ...  1.86602235  1.86602235
  1.86602235]
The best one (ground state) is : -7.784460280031249


# H<sub>4</sub>
Now we tackle H<sub>4</sub>, here what we are passing in the geometry is not the bond length, but rather the angle (in degrees). The bond length is fixed at $1.738\overset{\circ}{A}$

In [23]:
#Angle = 90 degrees
qubit_transf = "jw"
h4 = get_qubit_hamiltonian(mol='h4', geometry=90, basis='sto3g', qubit_transf=qubit_transf) #the geometry parameter takes angle instead of bond length for H4
print(h4)

-0.6427167034502693 [] +
-0.033820025472974885 [X0 X1 Y2 Y3] +
-0.03382002547394824 [X0 X1 Y4 Y5] +
-0.02090149320810566 [X0 X1 Y6 Y7] +
0.033820025472974885 [X0 Y1 Y2 X3] +
0.03382002547394824 [X0 Y1 Y4 X5] +
0.02090149320810566 [X0 Y1 Y6 X7] +
3.382506439134141e-07 [X0 Z1 X2 X3 Z4 Z5 Z6 X7] +
3.382506439134141e-07 [X0 Z1 X2 Y3 Z4 Z5 Z6 Y7] +
-0.012952358030619036 [X0 Z1 X2 X4 Z5 X6] +
-0.03426847228434699 [X0 Z1 X2 X5 Z6 X7] +
-0.03426847228434699 [X0 Z1 X2 Y5 Z6 Y7] +
-0.012952358030619032 [X0 Z1 Y2 Y4 Z5 X6] +
0.034268472284346985 [X0 Z1 Z2 X3 Y4 Z5 Z6 Y7] +
0.021316114253727956 [X0 Z1 Z2 X3 X5 X6] +
-0.034268472284346985 [X0 Z1 Z2 Y3 Y4 Z5 Z6 X7] +
0.021316114253727956 [X0 Z1 Z2 Y3 Y5 X6] +
-3.436313810401237e-07 [X0 Z1 Z2 Z3 X4 X5 Z6 X7] +
-3.436313810401237e-07 [X0 Z1 Z2 Z3 X4 Y5 Z6 Y7] +
-2.741320765142544e-07 [X0 Z1 Z2 Z3 Z4 Z5 X6] +
2.1423918919615432e-07 [X0 Z1 Z2 Z3 Z4 X6] +
-1.2939219184396936e-07 [X0 Z1 Z2 Z3 Z5 X6] +
-2.0991380574635217e-07 [X0 Z1 Z2 Z4 Z5 X6] +
1.283368

In [24]:
#Get additional information from tequila
xyz_data = get_molecular_data('h4', geometry=90, xyz_format=True)
basis='sto-3g'
h4_tq = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

#Taper Hamiltonian
h4_small = taper_hamiltonian(h4, n_spin_orbitals=2*h4_tq.n_orbitals, n_electrons=h4_tq.n_electrons, qubit_transf=qubit_transf)
print("The effective Hamiltonian:\n {}".format(h2o_small)) 

The effective Hamiltonian:
 -46.3574016028954 [] +
0.10505651008352154 [X0] +
-0.011299006073352824 [X0 X1 X2 Z3 Z4 Z6 Z7 X8] +
0.03522559376822524 [X0 X1 X2 Z8 Z9] +
0.0021883512242770473 [X0 X1 Z2 Z3 X4 Z5 Z7 Z8] +
-0.001664743498854376 [X0 X1 Z2 Z3 Z5 Z6 X7 Z8] +
-0.03172875970749076 [X0 X1 Y3 Y4] +
-0.005996185758110515 [X0 X1 Y3 Z4 Z6 Y7] +
0.011299006073352826 [X0 X1 Z3 Z4 Z6 Z7 X9] +
-0.005996185758110515 [X0 X1 X4 X6] +
0.0021883512242770473 [X0 X1 X4 Z6 Z7 Z8 Z9] +
0.03654446580389959 [X0 X1 X5] +
-0.024829490206145624 [X0 X1 Y6 Y7] +
-0.001664743498854376 [X0 X1 X7 Z8 Z9] +
-0.015996127256195754 [X0 X1 Y8 Y9] +
-0.011299006073352824 [X0 Y1 Y2 Z3 Z4 Z6 Z7 X8] +
0.03522559376822524 [X0 Y1 Y2 Z8 Z9] +
-0.0021883512242770473 [X0 Y1 Z2 Y3 Z4 Z5 Z7 Z8] +
0.0021883512242770473 [X0 Y1 Z2 Z3 Y4 Z5 Z7 Z8] +
0.0016647434988543758 [X0 Y1 Z2 Z3 Z5 Y6 Z7 Z8] +
-0.001664743498854376 [X0 Y1 Z2 Z3 Z5 Z6 Y7 Z8] +
0.03172875970749076 [X0 Y1 Y3 X4] +
0.005996185758110515 [X0 Y1 Y3 Z4 Z6 X7] +
-0

Yikes! This actually increased the number of qubits, I better use the first hamiltonian

In [25]:
h4_str = format(h4)
n_qubits = 8 # For h4


h4_ham_mat = str2mat(h4_str,n_qubits)

eigvals, _ = np.linalg.eigh(h4_ham_mat)
print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

print("The best one (ground state) is :",eigvals[0])


The eigenvalues in the effective Hamiltonian: 
 [-1.96946153e+00 -1.94221883e+00 -1.94221883e+00 -1.94221883e+00
 -1.82176626e+00 -1.71296389e+00 -1.70315529e+00 -1.70315528e+00
 -1.70315526e+00 -1.70315525e+00 -1.70315523e+00 -1.70315522e+00
 -1.67875947e+00 -1.67875945e+00 -1.67875924e+00 -1.67875921e+00
 -1.63511890e+00 -1.63511887e+00 -1.63511866e+00 -1.63511863e+00
 -1.49750968e+00 -1.49750968e+00 -1.49750968e+00 -1.49750968e+00
 -1.49750968e+00 -1.45878666e+00 -1.45878666e+00 -1.45878666e+00
 -1.45878666e+00 -1.44507275e+00 -1.44507275e+00 -1.44507275e+00
 -1.44507275e+00 -1.40519246e+00 -1.40519246e+00 -1.36904114e+00
 -1.36904113e+00 -1.36904111e+00 -1.36904110e+00 -1.36904108e+00
 -1.36904108e+00 -1.36276276e+00 -1.36276264e+00 -1.35780663e+00
 -1.35780663e+00 -1.34749355e+00 -1.34749355e+00 -1.34655283e+00
 -1.34655283e+00 -1.23920992e+00 -1.19744245e+00 -1.19744234e+00
 -1.18666547e+00 -1.18666547e+00 -1.17976830e+00 -1.17976830e+00
 -1.17903621e+00 -1.17903621e+00 -1.17903